In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk

import string
import re
from tqdm import tqdm

In [64]:
df = pd.read_csv('output/train_with_language.csv')
df.head(10)

,index,description,host_about,unlisted,description_language,host_about_language
0,1,"This is a shared mixed room in our hostel, wit...",Alojamento Local Registro: 20835/AL,0,en,pt
1,2,"O meu espaço fica perto de Parque Eduardo VII,...","I am friendly host, and I will try to always b...",1,pt,en
2,3,Trafaria’s House is a cozy and familiar villa ...,"I am a social person liking to communicate, re...",1,en,en
3,4,"Apartamento Charmoso no Chiado, Entre o Largo ...",Hello!\nI m Portuguese and i love to meet peop...,0,pt,en
4,5,Joli appartement en bordure de mer.<br /> 2 m...,Nous sommes une famille avec deux enfants de 1...,0,fr,fr
5,6,"IMPORTANT: In response to COVID-19, this prope...","Hi, we are Homing - a company that develops it...",0,en,en
6,7,This is my home that I rent out when I'm trave...,Globe trotter. I'm of Portuguese nationality w...,1,en,en
7,8,Find tranquility in this meticulously curated ...,I travel a lot and I love it. \nOriginally fro...,0,en,en
8,9,Charming apartment with one bedroom with doubl...,"Isabel & Helder, portugueses, parents of three...",0,en,en
9,10,Walk up original wooden stairs to the entrance...,Serviced holiday apartments casa in Azenhas do...,0,en,en


In [65]:
##### TEST ON A CHUNCK OF DATA
df = df.head(1000)

In [55]:
#!pip install spacy
#!python -m spacy download en_core_web_sm

In [66]:
import re
import spacy
nlp = spacy.load('en_core_web_sm')

In [67]:
def extract_number_of_people(text):
    # Regex pattern to match the number of people occurrences
    people_pattern = r'\b(\d+)\s*(people|guests|persons)\b'
    
    regex_matches = re.findall(people_pattern, text)
    numbers_of_people = [int(match[0]) for match in regex_matches]
    
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in ['QUANTITY', 'CARDINAL'] and any(word in ent.text for word in ['people', 'guests', 'persons']):
            try:
                numbers_of_people.append(int(ent.text))
            except ValueError:
                pass
    
    return numbers_of_people

In [75]:
def extract_number_of_people(text):
    pattern = r'\b\d+\s*(?:people?|guests?|persons?|friends?)\b'
    number_of_people = re.findall(pattern, text)
    
    return number_of_people

In [69]:
def extract_license_numbers(text):
    # Regex pattern to match license numbers in the format: "digits/digitsAL"
    license_pattern = r'\b\d+\/AL\b'    
    license_numbers = re.findall(license_pattern, text)
    
    return license_numbers

In [70]:
def extract_durations(text):
    time_pattern = r'\b\d+\s*(?:min|mins|minutes|min|hr|hrs|hour\b)'
    time_durations = re.findall(time_pattern, text)
    
    return time_durations

In [71]:
def extract_room_numbers(text):
    room_pattern = r'\b\d+\s*(?:bedrooms?|rooms?)\b'
    room_numbers = re.findall(room_pattern, text)
    return room_numbers

In [72]:
def extract_prices(text):
    euro_pattern = r'\b\d+\s*(?:euro(?:s)?|€)\b'
    prices = re.findall(euro_pattern, text)
    return prices

In [73]:
tqdm.pandas()
df['capacity'] = df['description'].progress_apply(lambda x: extract_number_of_people(x))
df['license'] = df['description'].progress_apply(lambda x: extract_license_numbers(x))
df['time_durations'] = df['description'].progress_apply(lambda x: extract_durations(x))
df['rooms'] = df['description'].progress_apply(lambda x: extract_room_numbers(x))
df['prices'] = df['description'].progress_apply(lambda x: extract_prices(x))

df[800:806]

100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 81207.85it/s]


,index,description,host_about,unlisted,description_language,host_about_language,capacity,license,time_durations,rooms,prices
800,801,Located on the center of Lisbon but on a very ...,"I'm Portuguese my husband is American, we spen...",0,en,en,[],[],[],[],[5 euros]
801,802,"Located in the authentic district of Graça, th...",Lisbonne Collection is a specialist company in...,1,en,en,[4 people],[],[],[],[]
802,803,"Welcome to Filipe, a bright and airy private h...",My name is Anita and I’m originally from Melbo...,1,en,en,[],[],[],[],[]
803,804,Blue ocean is a very quiet place about 800 met...,hello my name and Lisa\ni was born in germany ...,0,en,en,[],[],[],[],[]
804,805,Great house with beautiful deco and amazing vi...,We are pleased to present My Fair Lisbon as a ...,0,en,en,[],[35175/AL],[],[],[]
805,806,This is my son room. He is not allways with me...,Engenheiro e consultor apaixonado pelas tecnol...,1,en,pt,[],[],[10 min],[],[]


In [74]:
df.head(10)

,index,description,host_about,unlisted,description_language,host_about_language,capacity,license,time_durations,rooms,prices
0,1,"This is a shared mixed room in our hostel, wit...",Alojamento Local Registro: 20835/AL,0,en,pt,[],[],[5 min],[],[]
1,2,"O meu espaço fica perto de Parque Eduardo VII,...","I am friendly host, and I will try to always b...",1,pt,en,[],[],[],[],[]
2,3,Trafaria’s House is a cozy and familiar villa ...,"I am a social person liking to communicate, re...",1,en,en,[],[],"[20 min, 10 min]",[5 rooms],[]
3,4,"Apartamento Charmoso no Chiado, Entre o Largo ...",Hello!\nI m Portuguese and i love to meet peop...,0,pt,en,[],[91107/AL],[],[],[]
4,5,Joli appartement en bordure de mer.<br /> 2 m...,Nous sommes une famille avec deux enfants de 1...,0,fr,fr,[],[81300/AL],[2 min],[],[]
5,6,"IMPORTANT: In response to COVID-19, this prope...","Hi, we are Homing - a company that develops it...",0,en,en,"[4 people, 4 people]",[],[],"[2 bedrooms, 2 bedrooms]",[]
6,7,This is my home that I rent out when I'm trave...,Globe trotter. I'm of Portuguese nationality w...,1,en,en,[],[],[],[],[]
7,8,Find tranquility in this meticulously curated ...,I travel a lot and I love it. \nOriginally fro...,0,en,en,[],[69136/AL],[],[],[]
8,9,Charming apartment with one bedroom with doubl...,"Isabel & Helder, portugueses, parents of three...",0,en,en,[],[68527/AL],[],[1 bedroom],[]
9,10,Walk up original wooden stairs to the entrance...,Serviced holiday apartments casa in Azenhas do...,0,en,en,[],[80419/AL],[],[],[]


In [ ]:
def extract_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        if ent.label_ in ['QUANTITY', 'CARDINAL', 'ORDINAL', 'MONEY', 'DATE']:
            entities.append((ent.text, ent.label_))
    return entities